In [1]:
from agno.agent import Agent
from agno.models.ollama import Ollama
from pydantic import BaseModel
from pathlib import Path
from agno.reranker.base import Reranker
from agno.document.chunking.agentic import AgenticChunking
from agno.knowledge.document import DocumentKnowledgeBase, Document
from agno.knowledge.combined import CombinedKnowledgeBase
from agno.knowledge.text import TextKnowledgeBase
from agno.knowledge.docx import DocxKnowledgeBase, DocxReader
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.embedder.ollama import OllamaEmbedder
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.tools.file import FileTools
from agno.tools.duckduckgo import DuckDuckGoTools

# ollama pull openhermes llama3.1

In [2]:

vector_db = LanceDb(
    table_name="programs",
    uri="../tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=OllamaEmbedder(),
    reranker=Reranker()
)

chunking = AgenticChunking()

In [3]:
pdf_knowledge_base = PDFKnowledgeBase(
    path="../data/input",
    vector_db = vector_db
)

docx_knowledge_base = DocxKnowledgeBase(
    path="../data/input",
    vector_db = vector_db
)

txt_knowledge_base = TextKnowledgeBase(
    path="../data/input",
    vector_db = vector_db
)

knowledge_base = CombinedKnowledgeBase(
    sources=[
        pdf_knowledge_base,
        docx_knowledge_base,
        txt_knowledge_base,
    ],
    chunking_strategy=chunking,
    vector_db = vector_db
)


In [4]:
# knowledge_base.load(recreate=True)

In [5]:
for i in knowledge_base.document_lists:
    print(i)

INFO     Reading: Analisis y diseño de algoritmos

[Document(content='Análisis y diseño de algoritmos (3009430)', id='Analisis y diseño de algoritmos_1_1', name='Analisis y diseño de algoritmos', meta_data={'page': 1, 'chunk': 1, 'chunk_size': 41}, embedder=None, embedding=None, usage=None, reranking_score=None), Document(content='Objetivo del curso (oficial) Cualquiera con una experiencia mínima en algoritmos y programación sabe que para un mismo problema pueden existir innumerables algoritmos de solución. Tales soluciones pueden variar en diversos aspectos, siendo de particular interés la eficiencia en términos de su costo computacional. En el contexto de dicha eficiencia se puede hablar de algoritmos "mejores" que otros, siendo el objetivo principal de esta asignatura brindarle elementos al estudiante para analizar y diseñar algoritmos eficientes.', id='Analisis y diseño de algoritmos_2_1', name='Analisis y diseño de algoritmos', meta_data={'page': 2, 'chunk': 1, 'chunk_size': 529}, embedder=None, embedding=None, usage=None, reranki

INFO     Reading: Fundamentos de programacion

[Document(content='PRESENTACIÓN Y PROGRAMA DE ASIGNATURA FUNDAMENTOS DE PROGRAMACIÓN Juan Felipe Muñoz Fernández – jfmunozf@unal.edu.co • Código: 3010435 • Nombre: Fundamentos de programación • Grupos: 22, 23 y 24 • Número de créditos: 3 (Ingeniería de sistemas e informática) • Metodología ▪ Sesiones magistrales presenciales los jueves de 14:00 a 16:00 en el aula 302 del bloque 12 del núcleo El Volador. ▪ Sesiones prácticas presenciales los viernes en aula 204 del bloque 4 del núcleo del río: ▪ Grupo 22: viernes de 12:00 a 14:00 ▪ Grupo 23: viernes de 14:00 a 16:00 ▪ Grupo 24: viernes de 16:00 a 18:00 ▪ Asesorías por parte del docente en el horario de clase: jueves y viernes. ▪ Asesorías de monitores y auxiliares en el horario de las sesiones prácticas. • Temas del curso Unidad 1: Introducción ¿Qué es un algoritmo? ¿Qué es un programa de computadora y para qué sirve? ¿Por qué programar en Python? ¿Qué entornos de desarrollo se Python se pueden usar? Primer programa en Python, concepto 

In [6]:
description = "You are an academic content generation assistant. Based on the following university course syllabus, generate structured teaching materials, ensuring clarity, engagement, and alignment with best pedagogical practices. This material are going to be used by a professor to teach a course in [subject]."
instructions = [
    "You are a professional academic content creator, specialized in [subject].",
    "Analyze the syllabus and get the subject, topics, and learning objectives.",
    "Generate teaching materials for each topic, including lecture notes and practice problems in base of the user queries.",
    "Ensure the content follows best educational practices, including clear explanations, key concepts, real-world examples, and structured formatting.",
    "For each topic, provide a brief explanation, definitions, and key takeaways.",
    "Create conceptual and applied exercises with step-by-step solutions.",
    "If you don't know a term, search for it and provide a precise explanation.",
    "Generate the following teaching materials for each topic in base of the user queries:",
    "Lecture Notes: Clear explanations, definitions, real-world examples, and key takeaways.",  
    "Practice Problems: Conceptual and applied exercises with step-by-step solutions.",
    "Exams: Create a set of exam questions with solutions.",
    "If applicable, cite references or authoritative sources.",
    ]

In [7]:
agent = Agent(
    model=Ollama(id="llama3.1"),
    description=description,
    # instructions=instructions,
    tools=[
        FileTools(Path(), read_files = False, list_files = False), 
        DuckDuckGoTools(news = False)
        ],
    knowledge=knowledge_base, 
    show_tool_calls=True,
    markdown=True,
    search_knowledge=True,
    add_references=True,
    debug_mode=True
    )

In [ ]:
def query(course_name: str, content_type: str):
    return agent.print_response(
        f"Analyze the course '{course_name}' and identify the most relevant topics covered in a standard syllabus."
        f"Based on these topics, generate a '{content_type}' materials about the course search and explain them."
        "Explain each topic, provide definitions, and explain what each thing is."
        "Save the output in a file in the folder '../data/output' with a clear and descriptive filename.",
        markdown=True
    )

query("Fundamentos de programacion", "Notas de clase")

DEBUG    *********** Agent ID: f0b99c60-c403-464a-9afd-d13dad9ef7b3 ***********

DEBUG    *********** Session ID: 71c802c5-b7fa-4232-92c6-0c3779b573d8 ***********

DEBUG    *********** Agent Run Start: f2f806ea-e2d1-44c3-971a-9bdcad933261 ***********

DEBUG    Processing tools for model

DEBUG    Included function save_file from file_tools

...

DEBUG    ---------- Ollama Response Start ----------

DEBUG    ============== system ==============

DEBUG    You are an academic content generation assistant. Based on the following university course syllabus,      
         generate structured teaching materials, ensuring clarity, engagement, and alignment with best pedagogical 
         practices. This material are going to be used by a professor to teach a course in [subject].              
                                                                                                                   
         <additional_information>                                                                                  
         - Use markdown to format your answers.                                                                    
         </additional_information>

DEBUG    ============== user ==============

DEBUG    Analyze the course 'Fundamentos de programacion' and identify the most relevant topics covered in a       
         standard syllabus.Based on these topics, generate a 'Notas de clase' materials about the course search and
         explain them.Explain each topic, provide definitions, and explain what each thing is.Save the output in a 
         file in the folder '../data/output' with a clear and descriptive filename.

In [ ]:
query("Analisis y diseño de algoritmos", "Examen")